In [1]:
import re as re
import subprocess as sub
import sys as sys
import numpy as np

In [2]:
def not_real_val(output_val):
    if str(output_val) == str(1e400*0):
        NAN = True
    else:
        NAN = False
    return NAN

In [13]:
molecule_filename = sys.argv[1]
output_filename = molecule_filename + "_output.txt"

def write_energy_data(output_filename):
    i = 0
    output = open(output_filename, "w+")
    while True:
        convert_to_checkpoint = "formchk %s_%i.chk %s_%i.fchk"%(molecule_filename, i, molecule_filename, i)
        try:
            open(("%s_%i.chk"%(molecule_filename,i)))
        except IOError:
            break
        if (sub.call(convert_to_checkpoint, shell = True)) == 0:
            output_file = open(output_filename + "_%i.fchk"%i, "r")
            n_valence_electrons = 0
            MOs = []
            parse_line = True
            j = 0
            placeholder_reg_exp = re.compile("Number of electrons\s+I\s+(\d+)")
            for readout in output_file:
                if "Alpha Orbital Energies" in readout:
                    parse_line = True
                    j = 0
                if parse_line == True:
                    j += 1
                if parse_line == True and j >= 2:
                    try:
                        MOs.extend([float(s) for s in readout.split()])
                    except:
                        parse_line = False
                match_found = placeholder_reg_exp.search(readout)
                if type(match_found) != None:
                    n_valence_electrons = int(match_found.group(1))
        
            log_output = open(output_filename+"_%i.log"%i)
            reg_exp_exc = re.compile("Excited State\W+1:.*[^-]([+-]?\d+\.\d+)\W+eV")
            reg_exp_ground = re.compile("Excited State\W+1:.*[^-]([+-]?\d+\.\d+)\W+eV")
        	#matches the energy of the first exitation state
            NAN_flag = float('nan');
            for readout in log_output:
                match_found_exc = reg_exp_exc.search(readout)
                match_found_ground = reg_exp_ground.search(readout)
                if type(match_found_exc) != None:
                    first_exc_state = float(match_found_exc.group(1))
                if isNaN(first_exc_state) == True:
                    pass
                if type(match_found_ground) != None:
                    ground_state_ener = float(match_found_ground.group(1))
                if isNaN(ground_state_ener) == True:
                    pass
        

        elif (sub.call(convert_to_checkpoint, shell = True)) != 0:
            break

        output.write("%i %.10e %.10e\n"%(i,orbs[electrons/2-1]-orbs[electrons/2],first_exc_state,ground_state_ener)) 
    	#the homo/lumo orbitals are assumed to be at halfe the number of electrons
	
        i += 1
    output.close()
	
                
    